In [9]:
import numpy as np
import networkx as nx

def cal_all_pairs_minimax_path_matrix_by_algo_4(distance_matrix):
    N = len(distance_matrix)
    all_pairs_minimax_matrix = np.zeros((N, N))

    # Construct Minimum Spanning Tree (MST) from the graph
    MST = construct_MST_from_graph(distance_matrix)
    MST_edge_list = list(MST.edges(data='weight'))

    edge_node_list = [(edge[0], edge[1]) for edge in MST_edge_list]
    edge_weight_list = [edge[2] for edge in MST_edge_list]

    # Sort edges in descending order of weight
    edge_large_to_small_arg = np.argsort(edge_weight_list)[::-1]
    edge_weight_large_to_small = np.array(edge_weight_list)[edge_large_to_small_arg]
    edge_nodes_large_to_small = [edge_node_list[i] for i in edge_large_to_small_arg]

    # Iteratively remove edges and calculate tree nodes
    for i, edge_nodes in enumerate(edge_nodes_large_to_small):
        edge_weight = edge_weight_large_to_small[i]
        MST.remove_edge(*edge_nodes)

        tree1_nodes = list(nx.dfs_preorder_nodes(MST, source=edge_nodes[0]))
        tree2_nodes = list(nx.dfs_preorder_nodes(MST, source=edge_nodes[1]))

        for p1 in tree1_nodes:
            for p2 in tree2_nodes:
                all_pairs_minimax_matrix[p1, p2] = edge_weight
                all_pairs_minimax_matrix[p2, p1] = edge_weight

    # print(all_pairs_minimax_matrix)
    
    return all_pairs_minimax_matrix

def construct_MST_from_graph(distance_matrix):
    G = nx.Graph()
    N = len(distance_matrix)
    for i in range(N):
        for j in range(i + 1, N):
            G.add_edge(i, j, weight=distance_matrix[i][j])
    MST = nx.minimum_spanning_tree(G)
    return MST


In [10]:
def variant_of_Floyd_Warshall(adj_matrix):
    p = adj_matrix.copy()
    N = len(adj_matrix)

    for i in range(N):
        for j in range(N):
            if i != j:
                for k in range(N):
                    if i != k and j != k:
                        p[j, k] = min(p[j, k], max(p[j, i], p[i, k]))
    return p


In [11]:
import numpy as np
import time

# Utility function to create random adjacency matrices for undirected weighted graphs
def generate_random_graph(num_nodes, max_weight=100):
    adj_matrix = np.random.randint(1, max_weight + 1, size=(num_nodes, num_nodes))
    adj_matrix = np.triu(adj_matrix, 1)  # Keep only upper triangle
    adj_matrix += adj_matrix.T  # Make it symmetric for undirected graphs
    
    return adj_matrix

# Check correctness by comparing the two implementations
def check_correctness(adj_matrix):
    algo4_result = cal_all_pairs_minimax_path_matrix_by_algo_4(adj_matrix)
    fw_result = variant_of_Floyd_Warshall(adj_matrix)

    return np.allclose(algo4_result, fw_result)

# Measure execution time for a single test
def measure_execution_time(adj_matrix):
    start_algo4 = time.time()
    cal_all_pairs_minimax_path_matrix_by_algo_4(adj_matrix)
    end_algo4 = time.time()

    start_fw = time.time()
    variant_of_Floyd_Warshall(adj_matrix)
    end_fw = time.time()

    return end_algo4 - start_algo4, end_fw - start_fw

In [12]:
import numpy as np
import time

# Generate random graphs and calculate results
def generate_and_calculate_test_cases(test_sizes, num_tests=3, max_weight=100):
    test_cases = []
    for size in test_sizes:
        for test_num in range(num_tests):
            # Generate random graph
            adj_matrix = generate_random_graph(size, max_weight)

            # Calculate minimax path matrices using both algorithms
            # algo4_result = cal_all_pairs_minimax_path_matrix_by_algo_4(adj_matrix)
            # fw_result = variant_of_Floyd_Warshall(adj_matrix)

            # Verify correctness
            # correct = np.allclose(algo4_result, fw_result)

            # Save the test case details
            test_cases.append({
                "size": size,
                "test_num": test_num + 1,
                "adj_matrix": adj_matrix.tolist(),
                # "algo4_result": algo4_result.tolist(),
                # "fw_result": fw_result.tolist(),
                # "correct": correct
            })
    return test_cases

# Save test cases to a .py file
def save_test_cases_to_file(test_cases, file_name="test_cases.py"):
    with open(file_name, "w") as f:
        f.write("# Pre-calculated Test Cases for Minimax Path Problem\n\n")
        f.write("test_cases = [\n")
        for case in test_cases:
            f.write(f"    {case},\n")
        f.write("]\n")

# Generate, calculate, and save test cases
test_sizes = [5, 10, 20, 50, 100, 200, 500]
test_cases = generate_and_calculate_test_cases(test_sizes)
save_test_cases_to_file(test_cases)

print(f"Test cases successfully saved to 'test_cases.py'")


Test cases successfully saved to 'test_cases.py'


In [16]:
# Function to get test case graph
def get_test_case_graph(size):
    from test_cases import test_cases
    
    for test_case in test_cases:
        if test_case['size'] == size:
            return np.array(test_case['adj_matrix'])
    raise ValueError(f"No test case found for size {size}")

# Example usage
size = 50
adj_matrix = get_test_case_graph(size)
print(adj_matrix)

[[ 0  8 64 ... 63 98 18]
 [ 8  0 46 ... 76  9 72]
 [64 46  0 ... 30 11 55]
 ...
 [63 76 30 ...  0 24  1]
 [98  9 11 ... 24  0 13]
 [18 72 55 ...  1 13  0]]
